# Read in Data

In [1]:
# Import data.
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

df = pd.read_csv('watch_reviews.tsv', sep='\t', error_bad_lines=False)

[nltk_data] Downloading package punkt to /Users/judychen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/judychen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: exp

In [2]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [3]:
# Check missing values.
df.isnull().sum()

marketplace            0
customer_id            0
review_id              0
product_id             0
product_parent         0
product_title          2
product_category       0
star_rating            0
helpful_votes          0
total_votes            0
vine                   0
verified_purchase      0
review_headline        7
review_body          148
review_date            4
dtype: int64

In [4]:
# Remove if the review without review boday.
df.dropna(subset=['review_body'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [5]:
# Use the first 1000 data as training data.
data = df.loc[:999, 'review_body'].tolist()

# Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.

In [6]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br") #html <br>
stopwords.append("watch")

print("Stopwords that we use from nltk library: ")
print(stopwords)

Stopwords that we use from nltk library: 
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'no

In [7]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# Define functions to tokenize and stem reviews.
def tokenization_and_stemming(text):
    # exclude stop words and tokenize the document, generate a list of string
    tokens = []
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())
    
    # filter out any tokens not containing letters such as numeric tokens and raw punctuation.
    filtered_tokens = []
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
    
    # stemming.
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# Test function with the first review.
print(data[0])
print(tokenization_and_stemming(data[0]))


Absolutely love this watch! Get compliments almost every time I wear it. Dainty.
['absolut', 'love', 'get', 'compliment', 'almost', 'everi', 'time', 'wear', 'dainti']


# Term Frequency - Inverse Document Frequency

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Define vectorizer parameters, and use TfidfVectorizer to create tf-idf matrix
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000, min_df=0.01, stop_words='english', use_idf=True, 
                                tokenizer=tokenization_and_stemming, ngram_range=(1,1))

# Fit the vectorizer to synopses
tfidf_matrix = tfidf_model.fit_transform(data)
tfidf_matrix

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


<1000x239 sparse matrix of type '<class 'numpy.float64'>'
	with 6891 stored elements in Compressed Sparse Row format>

In [9]:
tfidf_matrix.toarray()

array([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [10]:
tfidf_matrix.todense()

matrix([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]])

In [11]:
# Save terms identified by TF-IDF
tf_selected_words = tfidf_model.get_feature_names()
print(tf_selected_words)

['abl', 'absolut', 'accur', 'actual', 'adjust', 'alarm', 'alreadi', 'alway', 'amaz', 'amazon', 'anoth', 'arm', 'arriv', 'automat', 'awesom', 'bad', 'band', 'batteri', 'beauti', 'best', 'better', 'big', 'bit', 'black', 'blue', 'bought', 'box', 'bracelet', 'brand', 'break', 'bright', 'broke', 'button', 'buy', 'ca', 'came', 'case', 'casio', 'chang', 'cheap', 'clasp', 'classi', 'clock', 'color', 'come', 'comfort', 'compliment', 'cool', 'cost', 'crown', 'crystal', 'dark', 'date', 'daughter', 'day', 'deal', 'definit', 'deliveri', 'design', 'dial', 'differ', 'difficult', 'disappoint', 'display', 'dress', 'durabl', 'easi', 'easili', 'end', 'everi', 'everyday', 'everyth', 'exact', 'excel', 'expect', 'expens', 'face', 'fair', 'far', 'fast', 'featur', 'feel', 'fell', 'fine', 'finish', 'fit', 'function', 'gave', 'gift', 'gold', 'good', 'got', 'great', 'hand', 'happi', 'hard', 'heavi', 'high', 'hold', 'honest', 'hope', 'hour', 'howev', 'husband', 'includ', 'instruct', 'invicta', 'issu', 'item', 'ke

# K-means Clustering

In [12]:
# k-means clustering
from sklearn.cluster import KMeans

# define number of clusters
num_clusters = 5
km = KMeans(n_clusters= num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [13]:
# Analyze K-means Result
# Create dataframe films from all of the input files.
product = {'review': df[:1000].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,2
1,I love this watch it keeps time wonderfully.,2
2,Scratches,0
3,"It works well on me. However, I found cheaper ...",0
4,Beautiful watch face. The band looks nice all...,0
5,"i love this watch for my purpose, about the pe...",2
6,"for my wife and she loved it, looks great and ...",3
7,I was about to buy this thinking it was a Swis...,0
8,Watch is perfect. Rugged with the metal &#34;B...,3
9,Great quality and build.<br />The motors are r...,0


In [14]:
# Number of reviews included in each cluster.
frame['cluster'].value_counts().to_frame()

,cluster
0,667
2,109
3,85
4,77
1,62


In [15]:
# km.cluster_centers_ denotes the importances of each items in centroid
km.cluster_centers_

array([[0.00565643, 0.00437174, 0.00382872, ..., 0.00653108, 0.01801031,
        0.01439971],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00784247,
        0.        ],
       [0.        , 0.04201663, 0.        , ..., 0.01244583, 0.01749062,
        0.00407598],
       [0.00379401, 0.        , 0.        , ..., 0.00262203, 0.        ,
        0.01821428],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00851693,
        0.        ]])

In [16]:
# Clustering result by K-means
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()

    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")

Cluster 0 words:look,like,work,band,time,beauti,
Cluster 0 reviews (667 reviews): 
Cluster 1 words:nice,price,look,realli,simpl,good,
Cluster 1 reviews (62 reviews): 
Cluster 2 words:love,wife,look,husband,beauti,absolut,
Cluster 2 reviews (109 reviews): 
Cluster 3 words:great,look,price,work,product,love,
Cluster 3 reviews (85 reviews): 
Cluster 4 words:good,product,qualiti,seller,price,love,
Cluster 4 reviews (77 reviews): 


# Topic Modeling - Latent Dirichlet Allocation

In [17]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [18]:
# document topic matrix for tfida_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output)

[[0.06004671 0.76120322 0.05969706 0.05946057 0.05959245]
 [0.08502113 0.66300099 0.08430606 0.08430205 0.08336976]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10000074 0.10049722 0.59852021 0.10000093 0.1009809 ]
 [0.06710348 0.73172289 0.06699369 0.06701126 0.06716867]
 [0.06741863 0.72174743 0.07024094 0.07006189 0.0705311 ]]


In [19]:
# Topics and words matrix
topic_word = lda.components_
print(topic_word)

[[ 2.44568829  4.06703652  0.20147694 ...  0.20422193  0.24742524
   4.37723324]
 [ 0.2025252   3.82679408  0.34728713 ...  5.55275052  1.26064608
   0.20171059]
 [ 1.94019121  0.20068566  2.27064566 ...  0.20277538 11.27641626
   3.75644527]
 [ 0.3066423   0.20112111  0.53396807 ...  0.2053792   0.30441805
   4.06092027]
 [ 0.20028007  0.20012545  0.20037729 ...  0.77057027  2.97248217
   0.20079609]]


In [20]:
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]
doc_names = ['Doc' + str(i) for i in range(len(data))]
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic
df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.06,0.76,0.06,0.06,0.06,1
Doc1,0.09,0.66,0.08,0.08,0.08,1
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.06,0.06,0.06,0.40,0.42,4
Doc4,0.09,0.05,0.65,0.04,0.18,2
Doc5,0.08,0.70,0.07,0.07,0.07,1
Doc6,0.06,0.07,0.28,0.06,0.52,4
Doc7,0.07,0.40,0.06,0.41,0.06,3
Doc8,0.76,0.05,0.05,0.09,0.05,0
Doc9,0.06,0.06,0.34,0.06,0.49,4


In [21]:
df_document_topic['topic'].value_counts().to_frame()

,topic
2,285
4,210
0,176
1,168
3,161


In [22]:
# topic word matrix
print(lda.components_)
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names
df_topic_words.head()

[[ 2.44568829  4.06703652  0.20147694 ...  0.20422193  0.24742524
   4.37723324]
 [ 0.2025252   3.82679408  0.34728713 ...  5.55275052  1.26064608
   0.20171059]
 [ 1.94019121  0.20068566  2.27064566 ...  0.20277538 11.27641626
   3.75644527]
 [ 0.3066423   0.20112111  0.53396807 ...  0.2053792   0.30441805
   4.06092027]
 [ 0.20028007  0.20012545  0.20037729 ...  0.77057027  2.97248217
   0.20079609]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,...,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,2.445688,4.067037,0.201477,0.202277,0.200999,1.854681,2.831140,1.226065,0.205149,4.212393,...,0.493502,1.024626,0.200169,1.568425,2.742879,0.510931,2.512676,0.204222,0.247425,4.377233
Topic1,0.202525,3.826794,0.347287,0.200742,0.469440,0.200842,0.719259,0.201891,0.201178,1.972618,...,1.332699,0.202622,1.105368,0.202609,0.202445,1.788617,1.080634,5.552751,1.260646,0.201711
Topic2,1.940191,0.200686,2.270646,4.559749,5.221345,1.491151,0.202089,1.609396,0.201474,0.204539,...,3.488420,0.212351,10.933831,1.592179,0.204197,3.978283,0.208500,0.202775,11.276416,3.756445
Topic3,0.306642,0.201121,0.533968,0.691861,0.200408,2.193234,0.202899,0.202612,9.957994,0.208854,...,0.200396,2.069946,0.200388,0.669077,1.220431,34.581226,0.200431,0.205379,0.304418,4.060920
Topic4,0.200280,0.200125,0.200377,0.201768,0.200979,0.200502,1.807980,1.600039,0.203040,0.204136,...,0.203346,0.207630,0.202642,0.201418,0.200556,0.202611,0.200853,0.770570,2.972482,0.200796


In [23]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,perfect,love,color,look,bought,husband,time,hand,cool,light,set,feel,tell,like,great
Topic 1,love,beauti,product,recommend,great,wear,time,buy,seller,qualiti,pleas,deliveri,water,look,excel
Topic 2,good,like,band,look,time,wrist,wife,size,realli,small,face,price,use,wear,qualiti
Topic 3,work,awesom,excel,use,amaz,broke,day,gift,batteri,week,want,arriv,time,stop,got
Topic 4,nice,great,look,expect,price,big,love,pretti,strap,good,comfort,littl,deal,thank,super
